## Ordering Basemaps via Orders API

In [12]:
import planet
import os
import requests
from requests.auth import HTTPBasicAuth
import json
import time
import pathlib

### Ordering a Basemap via an AOI  

step 1: find name of basemap on:
    planet explorer
    basemap api URL


step 2: create order

In [15]:
API_KEY = 'api key'
orders_url = 'https://api.planet.com/compute/ops/orders/v2' 
auth = HTTPBasicAuth('api key', '')
if os.environ.get(API_KEY, ''):
    API_KEY = os.environ.get(API_KEY, '')
else:
    API_KEY = API_KEY
    
# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

In [16]:
response = requests.get(orders_url, auth=session.auth)
response

<Response [200]>

In [17]:
order_params = {
   "name": "basemap order with geometry",
   "source_type": "basemaps",
   "order_type":"partial",
   "products": [
       {
           "mosaic_name": "global_monthly_2022_01_mosaic",
           "geometry":{
               "type": "Polygon",
               "coordinates":[
                   [
                       [ 4.607406, 52.353994 ],
                        [ 4.680005, 52.353994 ],
                        [ 4.680005, 52.395523 ],
                        [ 4.607406, 52.395523 ],
                        [ 4.607406, 52.353994 ]
                   ]
               ]
           }
      }
  ],
    "tools": [
      {"merge": {}},
      {"clip": {}}
      
  ],
 "delivery":{
      "google_cloud_storage":{
         "bucket":"notebook-practice",
         "credentials":"ewogICJ0eXBlIjogInNlcnZpY2VfYWNjb3VudCIsCiAgInByb2plY3RfaWQiOiAicGxhbmV0LWRldnJlbC1kZXYiLAogICJwcml2YXRlX2tleV9pZCI6ICIzMTBlM2ViNmFhY2E2MjAwODg3ZDk3YTg4Mjc2YzFmM2MwYjhhYWYwIiwKICAicHJpdmF0ZV9rZXkiOiAiLS0tLS1CRUdJTiBQUklWQVRFIEtFWS0tLS0tXG5NSUlFdmdJQkFEQU5CZ2txaGtpRzl3MEJBUUVGQUFTQ0JLZ3dnZ1NrQWdFQUFvSUJBUURKaGhjK0xrL09UeFJqXG5WZ3RuTjlERkU4SElveEFWaisrcWFlOWN5cWNIZW10RWkvdTJ5c0RDOVA2OXBmM2JrQ0pPQ2diT3V1L1NyUnJsXG4zdXQ3Z21NbkxBTmNvMzNDUGtieWNhMVpiVUxudnU0ZldwMGVQSndmZEtOR0NtZmtZKzF6YUZkVnp1blpqVzJ2XG5kZm9iTVU4TnN1QitLSWRBU0I2aGFxWlJyT1YzdElFQ0V6R24wcm5nOVhHRE51ampEb1FSdHBGMHJweDkxTTFWXG51d044SlRhYnFRcThXcXVzYzM4Y0E4Z1k1NHkzVkVuRmM1aEVyZ3podkZmVWhoTVpwWVJKd0IwckplQm15T1cyXG4xS08yWjdBVkt6UTFtSU9aNzY1MzlYT0RzRGxQb2VXSHdtOTE3UE1VYjgwTnUrTVZBdXp1aEE1eElnV0k4RXlZXG5NaXl5REtFZkFnTUJBQUVDZ2dFQVhSTjJscnlURlltR3RCNmdvNnF1M09jMElIUFgwTzJxMXYyWWxUWHdFVThrXG5UZ09OWGE2bGV5WG1wamNOV2FSbmc3Mnp5VFdqNEowZENFQithSXdmT1k5bGVocjAwWXNyLzN3cmdWc3FoSmkxXG5xSHVVaDh1Y25CYWw5b3FGMDRHbndSN1cvNFhTa2kxMUZJNGIxd1ZlS3NpTVVSaHM1MjZuM3d2QUI0Y2djTGwyXG5Lb2pzNk5tanIxU2VISmRVckpiMW5lK21Ec1RydTdiT1p4czFkMnEzb3ZNTDJTNlNuR2podlIwaFRRdmtvRzBZXG5vUGk5dFZaTUxoY3lyaEF0N0syajNMSmFQY0Q0OHZhTlZJTHJJU1czc2k4OXFqSmpCWVNBM2FSaWlmd1RzMzNFXG51Y1IvYVo2Nk1aNlI0aFI0aWNxTzcvbDFrOXpTRDBCeGJiWG1KNXl0T1FLQmdRRDNlaGFoK2ZwY0tML1daNDBOXG5DOVVpbzZkRWdMQlBYYkx4dkJudERrN0xIRTByaER2RXRYRnhmaGFtenlYUmROZWEydnRmUXJPRmtWUUNzOTIzXG45dU1SamRaTnNJdVFFU2x6clp3eDNZMTZqYlE1VGhyOTBVZFlTbjdMckRabTQzM3lYMjkwR1Y5YTdwa0EwdHZ6XG5zcHJiZjI2WWVOOWpQdHZOQWZ3U2NiNnlhd0tCZ1FEUWR0blV2SWY5Ym1QSjNpZ20vcVhMcGdWU0lDUkZiSCszXG4xN0dUSy82Uk5DN1A1cGhuaE55STAyVk1rblN0a0hldk5ob1Z3WmQySm9vc1JHWjBqVUJvMmxNZGcrMzZWdFA0XG45VHU2c09JdmsyN0N1L3dKVlNIVmRBZmVqTlZQTXhVSmxROXdVK1pmenpwU1JISVZlWWwxNmJiVmF4bnR2UFE2XG5MdWlmNGtBQkhRS0JnUURFbEF4S1dlRVBONmM2emtKeXJJVis0SFJMcTlWaEZkTEF4bVBwczhYTy9Rb25tUUJJXG4vci9BSVNJcEdyNjFwSlJuQTY5Tk9oU1Q4cDdERHREK0FuVFlTc3lDSmN5V2dhdGxvZkJDZy9TUDl1UEQ2VkR6XG5jVHUwM0dmZnJVb0JiSTlzNlhZRXlPcS90bXQvQmYrRmtzZ0R0elBJbGFzTWtLL1VKdEZ1YlQvSFJRS0JnUURQXG50cTBFR1RyVFhld1dWOExnWC9tcGVtcHcwMC93ME5STnhhRE5pUmVsYlQ1N3IwUUtXY3M0bHBLb0FvSTU4dW9lXG5iaVQ5cXFFVmUvOVBiblE4REFheXMxLy8xOVlIQy84NFlIbzhZa1Z0ckZTL3VubStmdEl3a3VYSWJjR1pvY2VEXG5mY0FkWjZ2enFWTnVaMzdaUVBtandOV0p2NmJkU05wOUl3SnRWUTM3VFFLQmdCNGJYU3NaajlTZFJ6Qk5PRms3XG5ZcUxTd1AyaDRaeFM5MFdsSW9WM0VrY3hmdjdLcHFSL3pCOVlNc21ZZjdJbUh6S3NPQU9QY1p1ODBha296bXRlXG5TY0VJYmh0azkwalJmc1AxaHpSRU9scGFkZ3l2VUJLdXhuNFVBNmw1RmhKbXg2WFVmS01vZnh1NURieFprYVJvXG5YVGRXNzhybm5ISjVJWExLMFA3QjkxdVNcbi0tLS0tRU5EIFBSSVZBVEUgS0VZLS0tLS1cbiIsCiAgImNsaWVudF9lbWFpbCI6ICJkZXZyZWwtZGV2LXRlc3Qtc2VydmljZWFjY291bnRAcGxhbmV0LWRldnJlbC1kZXYuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb20iLAogICJjbGllbnRfaWQiOiAiMTE0ODU4NzIxMDU1Mjk0NTY2NjU5IiwKICAiYXV0aF91cmkiOiAiaHR0cHM6Ly9hY2NvdW50cy5nb29nbGUuY29tL28vb2F1dGgyL2F1dGgiLAogICJ0b2tlbl91cmkiOiAiaHR0cHM6Ly9vYXV0aDIuZ29vZ2xlYXBpcy5jb20vdG9rZW4iLAogICJhdXRoX3Byb3ZpZGVyX3g1MDlfY2VydF91cmwiOiAiaHR0cHM6Ly93d3cuZ29vZ2xlYXBpcy5jb20vb2F1dGgyL3YxL2NlcnRzIiwKICAiY2xpZW50X3g1MDlfY2VydF91cmwiOiAiaHR0cHM6Ly93d3cuZ29vZ2xlYXBpcy5jb20vcm9ib3QvdjEvbWV0YWRhdGEveDUwOS9kZXZyZWwtZGV2LXRlc3Qtc2VydmljZWFjY291bnQlNDBwbGFuZXQtZGV2cmVsLWRldi5pYW0uZ3NlcnZpY2VhY2NvdW50LmNvbSIKfQo=",
         "path_prefix":"basemaps-to-cloud/"
      }


 }
}

In [18]:
ORDERS_API_URL = 'https://api.planet.com/compute/ops/orders/v2'
headers = {'content-type': 'application/json'}
paramRes = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers = {'content-type': 'application/json'})

print(paramRes.text)

{"_links":{"_self":"https://api.planet.com/compute/ops/orders/v2/bda6aaae-b2ff-48f9-a794-0d2c4342a9ea"},"created_on":"2022-11-18T22:39:28.111Z","delivery":{"google_cloud_storage":{"bucket":"notebook-practice","credentials":"<REDACTED>","path_prefix":"basemaps-to-cloud/"}},"error_hints":[],"id":"bda6aaae-b2ff-48f9-a794-0d2c4342a9ea","last_message":"Preparing order","last_modified":"2022-11-18T22:39:28.111Z","name":"basemap order with geometry","order_type":"partial","products":[{"geometry":{"coordinates":[[[4.607406,52.353994],[4.680005,52.353994],[4.680005,52.395523],[4.607406,52.395523],[4.607406,52.353994]]],"type":"Polygon"},"mosaic_name":"global_monthly_2022_01_mosaic"}],"source_type":"basemaps","state":"queued","tools":[{"merge":{}},{"clip":{}}]}



In [19]:
def place_order(order_params, auth):
    response = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = ORDERS_API_URL + '/' + order_id
    return order_url

In [20]:
order_url = place_order(order_params, session.auth)

<Response [202]>
36aa44c6-56ed-4c78-8570-105bebc13a6a


In [21]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=session.auth)
        response = r.json()
        print(response)
        
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
        # print(response)
        
poll_for_success(order_url, auth)

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/36aa44c6-56ed-4c78-8570-105bebc13a6a'}, 'created_on': '2022-11-18T22:39:28.810Z', 'delivery': {'google_cloud_storage': {'bucket': 'notebook-practice', 'credentials': '<REDACTED>', 'path_prefix': 'basemaps-to-cloud/'}}, 'error_hints': [], 'id': '36aa44c6-56ed-4c78-8570-105bebc13a6a', 'last_message': 'Preparing order', 'last_modified': '2022-11-18T22:39:28.810Z', 'name': 'basemap order with geometry', 'order_type': 'partial', 'products': [{'geometry': {'coordinates': [[[4.607406, 52.353994], [4.680005, 52.353994], [4.680005, 52.395523], [4.607406, 52.395523], [4.607406, 52.353994]]], 'type': 'Polygon'}, 'mosaic_name': 'global_monthly_2022_01_mosaic'}], 'source_type': 'basemaps', 'state': 'queued', 'tools': [{'merge': {}}, {'clip': {}}]}
queued
{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/36aa44c6-56ed-4c78-8570-105bebc13a6a'}, 'created_on': '2022-11-18T22:39:28.810Z', 'delivery': {'google_cloud_stora

In [22]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results']

KeyError: 'results'

In [ ]:
[r['name'] for r in results]

['basemaps-to-cloud/9e15388e-6c11-4945-b69c-3733e2e8d05d/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_metadata_merge_clip.json',
 'basemaps-to-cloud/9e15388e-6c11-4945-b69c-3733e2e8d05d/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_provenance_vector_merge_clip.zip',
 'basemaps-to-cloud/9e15388e-6c11-4945-b69c-3733e2e8d05d/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_ortho_udm_merge_clip.tif',
 'basemaps-to-cloud/9e15388e-6c11-4945-b69c-3733e2e8d05d/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_merge_clip.tif',
 'basemaps-to-cloud/9e15388e-6c11-4945-b69c-3733e2e8d05d/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_provenance_raster_merge_clip.tif',
 'basemaps-to-cloud/9e15388e-6c11-4945-b69c-3733e2e8d05d/manifest.json']

In [ ]:
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
download_results(results)

KeyError: 'location'

### Ordering a Basemap With a Set of Quad IDs

you can get the quad IDs from planet explorer or if you have the basemap, you can find the quad IDs like this:

In [ ]:
basemap_params = {
    'name__is': 'point_reyes_rolling_normalized_2020-04-30_mosaic'
}

In [ ]:
auth = HTTPBasicAuth('PLAK0962efb566a34514a56599994d2d0674', '')
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'
headers = {'content-type': 'application/json'}

basemap_params = {
    'name__is': 'point_reyes_rolling_normalized_2020-04-30_mosaic'
}

basemapServiceResponse = requests.get(url=BASEMAP_API_URL, params=basemap_params, auth=auth, headers=headers)

basemaps = basemapServiceResponse.raise_for_status()
if basemapServiceResponse.status_code != 204:
    basemaps = json.loads(basemapServiceResponse.text)
print(basemaps)

{'_links': {'_self': 'https://api.planet.com/basemaps/v1/mosaics?api_key=PLAK0962efb566a34514a56599994d2d0674'}, 'mosaics': [{'_links': {'_self': 'https://api.planet.com/basemaps/v1/mosaics/b7bed9c8-4dc7-4ecc-9968-d6b9fbcdcf3c?api_key=PLAK0962efb566a34514a56599994d2d0674', 'quads': 'https://api.planet.com/basemaps/v1/mosaics/b7bed9c8-4dc7-4ecc-9968-d6b9fbcdcf3c/quads?api_key=PLAK0962efb566a34514a56599994d2d0674&bbox={lx},{ly},{ux},{uy}', 'tiles': 'https://tiles.planet.com/basemaps/v1/planet-tiles/point_reyes_rolling_normalized_2020-04-30_mosaic/gmap/{z}/{x}/{y}.png?api_key=PLAK0962efb566a34514a56599994d2d0674'}, 'bbox': [-123.22265623283, 37.857507151805, -122.519531232927, 38.548165418546], 'coordinate_system': 'EPSG:3857', 'datatype': 'uint16', 'first_acquired': '2020-04-01T00:00:00.000Z', 'grid': {'quad_size': 4096, 'resolution': 4.77731426716}, 'id': 'b7bed9c8-4dc7-4ecc-9968-d6b9fbcdcf3c', 'interval': '1 day', 'item_types': ['PSScene4Band'], 'last_acquired': '2020-05-01T00:00:00.00

In [ ]:
mosaicID = basemaps['mosaics'][0]['id']
mosaicBbox = basemaps['mosaics'][0]['bbox']
mosaicBboxStr = ','.join(map(str, mosaicBbox))

quad_params = {
    'mosaic_id': mosaicID,
    'bbox': mosaicBboxStr,
}

In [ ]:
quads_url = "{}/{}/quads".format(BASEMAP_API_URL, mosaicID)
quadServiceResponse = requests.get(url=quads_url, params=quad_params, auth=auth, headers=headers)
quads = quadServiceResponse.json()
items = quads['items']

In [ ]:
quadIds = []
for i in items:
    quadId = i['id']
    quadIds.append(quadId)

In [ ]:
order_params = {
   "name":"basemap order with quad_ids",
   "source_type":"basemaps",
   "products":[
      {
         "mosaic_name":"point_reyes_rolling_normalized_2020-04-30_mosaic",
         "quad_ids": quadIds
      }
   ],
   #  "tools": [
   #    # {"merge": {}}
   #    # {"clip": {}},
   #  ],
    
   # "delivery":{
   #    "google_cloud_storage":{
   #       "bucket":"[your-orders-delivery-bucket]",
   #       "credentials":"[QUl6YVN5QWNlSzZFV1pGZ2I3Yklfc285UzNVTDRDanZfYlB1SDdVCg==]",
   #       "path_prefix":"[optionalsubfolder/]"
   #    }
   # }
}

In [ ]:
paramRes = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers=headers)

paramRes.raise_for_status()
if paramRes.status_code != 204:
    print(paramRes.text)

{"_links":{"_self":"https://api.planet.com/compute/ops/orders/v2/7ec9204d-5ecd-497a-8df4-944ae0c9f025"},"created_on":"2022-11-14T18:44:17.281Z","delivery":{"google_cloud_storage":{"bucket":"[your-orders-delivery-bucket]","credentials":"<REDACTED>","path_prefix":"[optionalsubfolder/]"}},"error_hints":[],"id":"7ec9204d-5ecd-497a-8df4-944ae0c9f025","last_message":"Preparing order","last_modified":"2022-11-14T18:44:17.281Z","name":"basemap order with quad_ids","products":[{"mosaic_name":"point_reyes_rolling_normalized_2020-04-30_mosaic","quad_ids":["323-1261","324-1261","325-1261","326-1261","323-1260","324-1260","325-1260","326-1260","323-1259","324-1259","325-1259","326-1259","323-1258","324-1258","325-1258","326-1258","323-1257","324-1257","325-1257","326-1257"]}],"source_type":"basemaps","state":"queued"}

